# Enefit - Predict Energy Behavior of Prosumers

The challenge in this competition is to predict the amount of electricity produced and consumed by Estonian energy customers who have installed solar panels. The dataset includes weather data, the relevant energy prices, and records of the installed photovoltaic capacity.

This is a forecasting competition using the time series API.

**Description**

The number of prosumers is rapidly increasing, and solving the problems of energy imbalance and their rising costs is vital. If left unaddressed, this could lead to increased operational costs, potential grid instability, and inefficient use of energy resources. If this problem were effectively solved, it would significantly reduce the imbalance costs, improve the reliability of the grid, and make the integration of prosumers into the energy system more efficient and sustainable. Moreover, it could potentially incentivize more consumers to become prosumers, knowing that their energy behavior can be adequately managed, thus promoting renewable energy production and use.

**About us**

Enefit is one of the biggest energy companies in Baltic region. As experts in the field of energy, we help customers plan their green journey in a personal and flexible manner as well as implement it by using environmentally friendly energy solutions.

At present, Enefit is attempting to solve the imbalance problem by developing internal predictive models and relying on third-party forecasts. However, these methods have proven to be insufficient due to their low accuracy in forecasting the energy behavior of prosumers. The shortcomings of these current methods lie in their inability to accurately account for the wide range of variables that influence prosumer behavior, leading to high imbalance costs. By opening up the challenge to the world's best data scientists through the Kaggle platform, Enefit aims to leverage a broader pool of expertise and novel approaches to improve the accuracy of these predictions and consequently reduce the imbalance and associated costs.

**Evaluation**

Submissions are evaluated on the Mean Absolute Error (MAE) between the predicted return and the observed target. The formula is given by:

𝑀𝐴𝐸=1𝑛∑𝑖=1𝑛|𝑦𝑖−𝑥𝑖|
MAE = 1n∑i=1n|yi-xi|

Where:
* 𝑛 is the total number of data points.
* 𝑦𝑖 is the predicted value for data point i.
* 𝑥𝑖 is the observed value for data point i.

**Submitting**

You must submit to this competition using the provided python time-series API, which ensures that models do not peek forward in time. To use the API, follow the template in this [notebook](https://www.kaggle.com/code/sohier/enefit-basic-submission-demo).

**Timeline**

This is a future data prediction competition with an active training phase and a second period where selected submissions will be evaluated against future ground truth data.

*Training Timeline*

* November 1, 2023 - Start Date.
* January 24, 2024 - Entry Deadline. You must accept the competition rules before this date in order to compete.
* January 24, 2024 - Team Merger Deadline. This is the last day participants may join or merge teams.
* January 31, 2024 - Final Submission Deadline.

All deadlines are at 11:59 PM UTC on the corresponding day unless otherwise noted. The competition organizers reserve the right to update the contest timeline if they deem it necessary.

*Prediction Timeline:*

Starting after the final submission deadline there will be periodic updates to the leaderboard to reflect future data updates that will be evaluated against selected submissions. We anticipate 1-3 interim updates before the final evaluation.

* April 30, 2024 - Competition End Date

**Prizes**

* 1st Place - $ 15,000
* 2nd Place - $ 10,000
* 3rd Place - $ 8,000
* 4th Place - $ 7,000
* 5th Place - $ 5,000
* 6th Place - $ 5,000

**Code Requirements**

Submissions to this competition must be made through Notebooks. In order for the "Submit" button to be active after a commit, the following conditions must be met:

* CPU Notebook <= 9 hours run-time
* GPU Notebook <= 9 hours run-time
* Internet access disabled
* Freely & publicly available external data is allowed, including pre-trained models
* Submission file must be named submission.csv and be generated by the API.

Please see the [Code Competition FAQ](https://www.kaggle.com/docs/competitions#notebooks-only-FAQ) for more information on how to submit. And review the [code debugging doc](https://www.kaggle.com/code-competition-debugging) if you are encountering submission errors.

### Load Workspace

In [ ]:
import re
import datetime as dt
import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('fivethirtyeight')
import seaborn as sns
import opendatasets as od
import kaggle
import zipfile
import io
import json
import warnings

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose, STL
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.gofplots import qqplot
from statsmodels.tsa.stattools import adfuller
from tqdm import notebook
from itertools import product
from typing import Union

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, classification_report

### Load the Dataset

In [ ]:
def list_files_in_zip(zip_file_path):
    zip_files = list()
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        for file in file_list:
            zip_files.append(file)
    return zip_files

zip_file_path = 'predict-energy-behavior-of-prosumers.zip'

enefit_files = list_files_in_zip(zip_file_path)
enefit_files

In [ ]:
def read_csv_from_zip(zip_file_path, csv_file_name):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        with zip_ref.open(csv_file_name) as file:
            df = pd.read_csv(io.TextIOWrapper(file))
            return df

def read_json_from_zip(zip_file_path, json_file_name):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        with zip_ref.open(json_file_name) as file:
            data = json.load(file)
            df = pd.DataFrame(data, index=range(len(data)))
            return df

def clean_date(df):
    date_cols = ['date', 'datetime', 'forecast_date', 'origin_date', 'forecast_datetime', 'origin_datetime']
    for col in df.columns:
        if col in date_cols:
            df[col] = pd.to_datetime(df[col])
    return df

In [ ]:
enefit_dict = dict()
keys = [
    'client', 'electricity_prices', 
    'weather_station_to_county_mapping', 'county_id_to_name_map', 
]

In [ ]:
for key in keys:
    if key + '.csv' in enefit_files:
        csv_file_name = key + '.csv'
        enefit_dict[key] = read_csv_from_zip(zip_file_path, csv_file_name)
    elif key + '.json' in enefit_files:
        json_file_name = key + '.json'
        enefit_dict[key] = read_json_from_zip(zip_file_path, json_file_name)

enefit_dict['county_id_to_name_map'] = enefit_dict['county_id_to_name_map'].iloc[0].T

for key in keys[0:2]:
    enefit_dict[key] = clean_date(enefit_dict[key])

### Data Exploration

In [ ]:
deep_colors = [
    '#4C72B0', '#55A868', '#C44E52',
    '#8172B2', '#CCB974', '#64B5CD'
]

**Client**

The features in this dataset are:
* product_type: ID code with the following mapping of codes to contract types: {0: "Combined", 1: "Fixed", 2: "General service", 3: "Spot"}
* county: An ID code for the county. See county_id_to_name_map.json for the mapping of ID codes to county names.
* eic_count: The aggregated number of consumption points (EICs - European Identifier Code).
* installed_capacity: Installed photovoltaic solar panel capacity in kilowatts.
* is_business: Boolean for whether or not the prosumer is a business.
* date
* data_block_id: All rows sharing the same `data_block_id` will be available at the same forecast time. This is a function of what information is available when forecasts are actually made, at 11 AM each morning. For example, if the forecast weather `data_block_id` for predictions made on October 31st is 100 then the historic weather data_block_id for October 31st will be 101 as the historic weather data is only actually available the next day.

In [7]:
enefit_dict['client'].head()

,product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
0,1,0,108,952.89,0,2021-09-01,2
1,2,0,17,166.40,0,2021-09-01,2
2,3,0,688,7207.88,0,2021-09-01,2
3,0,0,5,400.00,1,2021-09-01,2
4,1,0,43,1411.00,1,2021-09-01,2


In [8]:
pc_map = {0: "Combined", 1: "Fixed", 
          2: "General service", 3: "Spot"}

enefit_dict['client'] = enefit_dict['client'].assign(
    product_type_map=lambda x: x.product_type.map(pc_map),
    county_map=lambda x: x.county.astype(str).map(enefit_dict['county_id_to_name_map'].str.title().to_dict())
)

Most electricity demand occurs in the spot market. 

In [ ]:
fig, ax = plt.subplots()
enefit_dict['client'].product_type_map.value_counts().plot(
    kind='barh', xlabel='Frequency', 
    ylabel='Product Type',
    title='Distribution of Product Types',
    ax=ax
).invert_yaxis()
ax.grid(alpha=.25)
plt.show()

In [ ]:
fig, ax = plt.subplots()
enefit_dict['client'].county_map.value_counts().plot(
    kind='barh', xlabel='Frequency',
    ylabel='County',
    title='Distribution of Counties',
    ax=ax
).invert_yaxis()
ax.grid(alpha=.25)
plt.show()

**Installed Capacity**

Harjumaa county has the highest total installed capacity.

In [ ]:
plot_order = enefit_dict['client'].groupby(by='county_map').agg({'installed_capacity':'sum'}).sort_values(by='installed_capacity', ascending=False).index.values

fig, ax = plt.subplots(figsize=(8, 6))
sns.barplot(
    data=enefit_dict['client'],
    x='installed_capacity',
    y='county_map',
    estimator='sum',
    orient='h',
    order=plot_order,
    ax=ax
)
ax.grid(alpha=.25)

ax.set_xlabel('Total Installed Capacity')
ax.set_ylabel('Counties')
ax.set_title('Total Installed Capacity (kW) across Counties')

plt.show()

For most counties, installed capacity for businesses is higher than capacity for non-businesses.

In [ ]:
df = enefit_dict['client'].groupby(by=['county_map', 'is_business']).installed_capacity.sum().unstack().rename(columns={0:'not_business', 1:'business'})

plt.figure(figsize=(10, 6))
plt.bar(df.index, df.not_business, width=0.4, align='center', label='Not Business')
plt.bar(df.index, df.business, width=0.4, align='edge', label='Business')

plt.xlabel('Counties')
plt.ylabel('Total Installed Capacity (kW)')
plt.title('Total Installed Capacity Segmented by Business / Not Business')
plt.legend()
plt.xticks(rotation=90)
plt.grid(alpha=.25)
plt.tight_layout()
plt.show()


Most activity is in the spot market, where business clients are more popular. The second most popular market is the fixed market, where non-business clients dominate. There is nearly no activity in the general service market and the combined market services only business consumers.

Developing a model to reduce energy imbalance costs will have to prioritize the spot and fixed markets, which make up nearly all products sold.

In [ ]:
plt.figure(figsize=(10, 8))
g = sns.catplot(
    data=enefit_dict['client'],
    kind="bar",
    orient='h',
    y='county_map',
    x='installed_capacity',
    hue='is_business',
    col='product_type_map',
    col_wrap=2,
    height=4,
    estimator=sum
)

g.set_axis_labels('County', 'Installed Capacity')
g.set_titles('Product Type: {col_name}')

new_labels = ['Non-business', 'Business']
for ax in g.axes.flat:
    handles, _ = ax.get_legend_handles_labels()
    ax.legend(handles, new_labels, loc='center left', bbox_to_anchor=(1, 0.5))

# Remove the previous legend
g._legend.remove()

plt.subplots_adjust(top=0.9)
g.fig.suptitle('Total Installed Capacity (kW) by Product Type & Business Segments')

plt.show()


Let's drill down to general service & combined to see them clearer:

In [ ]:
df = enefit_dict['client'].query(
    'product_type_map=="General service"'
).groupby(by=['county_map', 'is_business']).installed_capacity.sum().unstack().rename(columns={0:'not_business', 1:'business'})

plt.figure(figsize=(10, 6))
plt.bar(df.index, df.not_business, width=0.4, align='center', label='Not Business')
plt.bar(df.index, df.business, width=0.4, align='edge', label='Business')

plt.xlabel('Counties')
plt.ylabel('Total Installed Capacity (kW)')
plt.title('Total Installed Capacity (General service) Segmented by Business / Not Business')
plt.legend()
plt.xticks(rotation=45)
plt.grid(alpha=.25)
plt.show()


In [ ]:
df = enefit_dict['client'].query(
    'product_type_map=="Combined"'
).groupby(by=['county_map', 'is_business']).installed_capacity.sum().unstack().rename(columns={0:'not_business', 1:'business'})
df['not_business'] = 0

plt.figure(figsize=(10, 6))
plt.bar(df.index, df.not_business, width=0.4, align='center', label='Not Business')
plt.bar(df.index, df.business, width=0.4, align='edge', label='Business')

plt.xlabel('Counties')
plt.ylabel('Total Installed Capacity (kW)')
plt.title('Total Installed Capacity (Combined) Segmented by Business / Not Business')
plt.legend()
plt.xticks(rotation=45)
plt.grid(alpha=.25)
plt.show()


In [ ]:
# Resample the data to monthly frequency and calculate month-to-month change
# Resampling to month-end frequency
monthly_data = enefit_dict['client'].set_index('date').installed_capacity.resample('M').last()  

# Calculate the absolute increase
monthly_data.name = monthly_data.name.strip()
month_to_month_change = monthly_data.diff()

# Calculate the average increase
avg_month_to_month_change = month_to_month_change.mean()

print(f"Average month-on-month increase in Installed Capacity: {avg_month_to_month_change:.2f}kW")

In [ ]:
df = enefit_dict['client'].set_index('date').installed_capacity.resample('D').sum()
fig, ax = plt.subplots()

ax.plot(df)
ax.set_xlabel('Date')
ax.set_ylabel('Installed Capacity (kW)')
ax.set_title('Total Daily Installed Capacity Over Time')

fig.autofmt_xdate()
plt.tight_layout()
plt.show()

In [ ]:
sns.lineplot(
    x='date', y='installed_capacity', data=enefit_dict['client']
)
plt.xlabel('Date')
plt.ylabel('Installed Capacity')
plt.title('Time Series Plot of Installed Capacity (kW)')
plt.xticks(rotation=45)
plt.grid(alpha=.3)
plt.show()

Let's consider if installed capacity can be modeled using time series forecasting.

The time series for installed capaity has a clear trend. However, seasonality is not clear:

In [ ]:
decomposition = STL(enefit_dict['client'].set_index('date').installed_capacity, period=365).fit()

fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, ncols=1, sharex=True, figsize=(10,8))

ax1.plot(decomposition.observed, color=deep_colors[0])
ax1.set_ylabel('Observed')

ax2.plot(decomposition.trend, color=deep_colors[1])
ax2.set_ylabel('Trend')

ax3.plot(decomposition.seasonal, color=deep_colors[2])
ax3.set_ylabel('Seasonal')

ax4.plot(decomposition.resid, color=deep_colors[3])
ax4.set_ylabel('Residuals')

fig.autofmt_xdate()
plt.tight_layout()

plt.show()

Total Daily Installed:

In [ ]:
decomposition = STL(enefit_dict['client'].set_index('date').installed_capacity.resample('D').sum(), period=365).fit()

fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, ncols=1, sharex=True, figsize=(10,8))

ax1.plot(decomposition.observed, color=deep_colors[0])
ax1.set_ylabel('Observed')

ax2.plot(decomposition.trend, color=deep_colors[1])
ax2.set_ylabel('Trend')

ax3.plot(decomposition.seasonal, color=deep_colors[2])
ax3.set_ylabel('Seasonal')

ax4.plot(decomposition.resid, color=deep_colors[3])
ax4.set_ylabel('Residuals')

fig.autofmt_xdate()
plt.tight_layout()

plt.show()

Test for stationarity:

Due to the nature of the data, we'll resample the data to add up or average all records for a particular date. Then we'll use first order differencing to make time series stationary if it is not.

Total Daily Installed:

In [ ]:
df = enefit_dict['client'].set_index('date').installed_capacity.resample('D').sum()
ad_fuller_result = adfuller(df)

print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

In [ ]:
df_diff = np.diff(df, n=1)

ad_fuller_result = adfuller(df_diff)
print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

Testing for Autocorrelation & Partial Autocorrelation: 

Visually, there seems to be no lags after the first lag for both autoregressive and moving average metrics in the installed capacity time series. This implies that the series is a random walk. 

A random walk is a process in which there is an equal chance of going up or down by a random number. Random walks often expose long periods where a positive or negative trend can be observed. They are also often accompanied by sudden changes in direction.

We can only use naive forecasting methods with random walks. This means that the installed capacity cannot be modeled using an AutoRegressive Integrated Moving Average (ARIMA) model. Naive forecasting methods include mean, last value, last seasonal value, drift.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
plot_acf(df_diff, lags=10, ax=ax1)
plot_pacf(df_diff, lags=10, ax=ax2)
plt.tight_layout()
plt.show()

With a random walk, we can only forecast using naive methods such as mean, last value, last seasonal value, etc. We'll consider using linear regression to model time and serial dependence for forecasting.

In [ ]:
df = df.to_frame()
df['time'] = np.arange(len(df.index)) # set time steps from index
df['lag_1'] = df.installed_capacity.shift(1) # First lag
df = df.reindex(columns=['installed_capacity', 'time', 'lag_1'])

df.installed_capacity = df.installed_capacity.astype(float)
df.lag_1 = df.lag_1.astype(float)

df.head()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 7))

ax1.plot(df.time, df.installed_capacity, color=deep_colors[0])
sns.regplot(
    x='time', y='installed_capacity', data=df,
    ci=None, scatter_kws=dict(color=deep_colors[1]), ax=ax1
)
ax1.set_title('Time Plot of Installed Capacity')

sns.regplot(
    x='lag_1', y='installed_capacity', data=df,
    ci=None, scatter_kws=dict(color=deep_colors[0]), ax=ax2
)
ax2.set_aspect('equal')
ax2.set_title('Lag Plot of Installed Capacity')

plt.tight_layout()
plt.show()

OLS Models:

In [ ]:
formula = 'installed_capacity ~ time'
model = smf.ols(formula, data=df)
results = model.fit()
print(results.summary())

In [ ]:
formula = 'installed_capacity ~ lag_1'
model = smf.ols(formula, data=df)
results = model.fit()
print(results.summary())

The models created are approximately: 
- `Installed Capacity = 104.8 * time + 6234`
- `Installed Capacity = 1 * lag_1 + -203.77`

We can use these formulas to predict future values. We'll use these models along with naive forecasts to predict future installed capacity.

In [ ]:
def rolling_forecast(
        df: pd.DataFrame, train_len: int, horizon: int,
        window: int, method: str, pdq=(0,0,0)
) -> list:

    total_len = train_len + horizon

    if method == 'mean':
        pred_mean = []

        for i in range(train_len, total_len, window):
            # mean = np.mean(df[:i].values)
            mean = np.mean(df[:i])
            pred_mean.extend(mean for _ in range(window))

        return pred_mean

    elif method == 'last':
        pred_last_value = []

        for i in range(train_len, total_len, window):
            # last_value = df[:i].iloc[-1].values[0]
            last_value = df[:i].iloc[-1]#.values[0]
            pred_last_value.extend(last_value for _ in range(window))

        return pred_last_value

In [ ]:
df_train = df.installed_capacity.loc[:'2023-04-30']
df_test = df.installed_capacity.loc['2023-05-01':]

train_len = len(df_train)
horizon = len(df_test)
window = 1

pred_mean = rolling_forecast(df['installed_capacity'], train_len, horizon, window, 'mean')
pred_last_value = rolling_forecast(df['installed_capacity'], train_len, horizon, window, 'last')

In [ ]:
y_train = df.installed_capacity.loc[:'2023-04-30']
y_test = df.installed_capacity.loc['2023-05-01':]

X_time_train = df.loc[:'2023-04-30', ['time']]
X_time_test = df.loc['2023-05-01':, ['time']]

model = LinearRegression()
model.fit(X_time_train, y_train)
pred_timestep = pd.Series(model.predict(X_time_test), index=X_time_test.index)

In [ ]:
X_lag_train = df.loc[:'2023-04-30', ['lag_1']].dropna()
X_lag_test = df.loc['2023-05-01':, ['lag_1']].dropna()
y_train_lag, X_lag_train = y_train.align(X_lag_train, join='inner')

model = LinearRegression()
model.fit(X_lag_train, y_train_lag)
pred_lag = pd.Series(model.predict(X_lag_test), index=X_lag_test.index)

In [ ]:
df_test = df_test.to_frame().copy()
df_test.loc[:, 'pred_mean'] = pred_mean
df_test = df_test.copy()
df_test.loc[:, 'pred_last_value'] = pred_last_value
df_test = df_test.copy()
df_test.loc[:, 'pred_timestep'] = pred_timestep
df_test = df_test.copy()
df_test.loc[:, 'pred_lag'] = pred_lag

df_test.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df_train.loc['2023-04':])
ax.plot(df_test['installed_capacity'], 'b-', label='actual')
ax.plot(df_test['pred_mean'], 'g:', label='mean')
ax.plot(df_test['pred_last_value'], 'r-.', label='last')
ax.plot(df_test['pred_timestep'], 'k--', label='timestep')
ax.plot(df_test['pred_lag'], 'y--', label='lag_1')

ax.legend(loc=2, fontsize='small')

ax.set_xlabel('Date')
ax.set_ylabel('Installed Capacity')
ax.set_title('Installed Capacity with Forecasts')

fig.autofmt_xdate()
plt.tight_layout()
plt.show()

Average Daily Installed:

In [ ]:
decomposition = STL(enefit_dict['client'].set_index('date').installed_capacity.resample('D').mean(), period=365).fit()

fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, ncols=1, sharex=True, figsize=(10,8))

ax1.plot(decomposition.observed, color=deep_colors[0])
ax1.set_ylabel('Observed')

ax2.plot(decomposition.trend, color=deep_colors[1])
ax2.set_ylabel('Trend')

ax3.plot(decomposition.seasonal, color=deep_colors[2])
ax3.set_ylabel('Seasonal')

ax4.plot(decomposition.resid, color=deep_colors[3])
ax4.set_ylabel('Residuals')

fig.autofmt_xdate()
plt.tight_layout()

plt.show()

Test for Stationarity:

In [ ]:
df = enefit_dict['client'].set_index('date').installed_capacity.resample('D').mean()
ad_fuller_result = adfuller(df)

print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

In [ ]:
df_diff = np.diff(df, n=1)

ad_fuller_result = adfuller(df_diff)
print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

Testing for Autocorrelation & Partial Autocorrelation: 

Visually, there seems to be no lags after the first lag for both autoregressive and moving average metrics in the installed capacity time series. This implies that the series is a random walk.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
plot_acf(df_diff, lags=10, ax=ax1)
plot_pacf(df_diff, lags=10, ax=ax2)
plt.tight_layout()
plt.show()

In [ ]:
df = df.to_frame()
df['time'] = np.arange(len(df.index)) # set time steps from index
df['lag_1'] = df.installed_capacity.shift(1) # First lag
df = df.reindex(columns=['installed_capacity', 'time', 'lag_1'])

df.installed_capacity = df.installed_capacity.astype(float)
df.lag_1 = df.lag_1.astype(float)

df.head()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 7))

ax1.plot(df.time, df.installed_capacity, color=deep_colors[0])
sns.regplot(
    x='time', y='installed_capacity', data=df,
    ci=None, scatter_kws=dict(color=deep_colors[1]), ax=ax1
)
ax1.set_title('Time Plot of Installed Capacity')

sns.regplot(
    x='lag_1', y='installed_capacity', data=df,
    ci=None, scatter_kws=dict(color=deep_colors[0]), ax=ax2
)
ax2.set_aspect('equal')
ax2.set_title('Lag Plot of Installed Capacity')

plt.tight_layout()
plt.show()

OLS Models:

In [ ]:
formula = 'installed_capacity ~ time'
model = smf.ols(formula, data=df)
results = model.fit()
print(results.summary())

In [ ]:
formula = 'installed_capacity ~ lag_1'
model = smf.ols(formula, data=df)
results = model.fit()
print(results.summary())

The models created are approximately: 
- `Installed Capacity = 1.46 * time + 984.24`
- `Installed Capacity = 1 * lag_1 + -5.36`

We can use these formulas to predict future values. We'll use these models along with naive forecasts to predict future installed capacity.

In [ ]:
df_train = df.installed_capacity.loc[:'2023-04-30']
df_test = df.installed_capacity.loc['2023-05-01':]

train_len = len(df_train)
horizon = len(df_test)
window = 1

pred_mean = rolling_forecast(df['installed_capacity'], train_len, horizon, window, 'mean')
pred_last_value = rolling_forecast(df['installed_capacity'], train_len, horizon, window, 'last')

In [ ]:
y_train = df.installed_capacity.loc[:'2023-04-30']
y_test = df.installed_capacity.loc['2023-05-01':]

X_time_train = df.loc[:'2023-04-30', ['time']]
X_time_test = df.loc['2023-05-01':, ['time']]

model = LinearRegression()
model.fit(X_time_train, y_train)
pred_timestep = pd.Series(model.predict(X_time_test), index=X_time_test.index)

In [ ]:
X_lag_train = df.loc[:'2023-04-30', ['lag_1']].dropna()
X_lag_test = df.loc['2023-05-01':, ['lag_1']].dropna()
y_train_lag, X_lag_train = y_train.align(X_lag_train, join='inner')

model = LinearRegression()
model.fit(X_lag_train, y_train_lag)
pred_lag = pd.Series(model.predict(X_lag_test), index=X_lag_test.index)

In [ ]:
df_test = df_test.to_frame().copy()
df_test.loc[:, 'pred_mean'] = pred_mean
df_test = df_test.copy()
df_test.loc[:, 'pred_last_value'] = pred_last_value
df_test = df_test.copy()
df_test.loc[:, 'pred_timestep'] = pred_timestep
df_test = df_test.copy()
df_test.loc[:, 'pred_lag'] = pred_lag

df_test.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df_train.loc['2023-04':])
ax.plot(df_test['installed_capacity'], 'b-', label='actual')
ax.plot(df_test['pred_mean'], 'g:', label='mean')
ax.plot(df_test['pred_last_value'], 'r-.', label='last')
ax.plot(df_test['pred_timestep'], 'k--', label='timestep')
ax.plot(df_test['pred_lag'], 'y--', label='lag_1')

ax.legend(loc=2, fontsize='small')

ax.set_xlabel('Date')
ax.set_ylabel('Installed Capacity')
ax.set_title('Installed Capacity with Forecasts')

fig.autofmt_xdate()
plt.tight_layout()
plt.show()

**Consumption Points**

In [ ]:
plot_order = enefit_dict['client'].groupby(by='county_map').agg({'eic_count':'sum'}).sort_values(by='eic_count', ascending=False).index.values

fig, ax = plt.subplots(figsize=(8, 6))
sns.barplot(
    data=enefit_dict['client'],
    x='eic_count',
    y='county_map',
    estimator='sum',
    orient='h',
    order=plot_order,
    ax=ax
)
ax.grid(alpha=.25)

ax.set_xlabel('Total Consumption Points')
ax.set_ylabel('Counties')
ax.set_title('Total Consumption Points across Counties')

plt.show()

For all counties, consumption points are higher for non-business users than for business users. 

In [ ]:
df = enefit_dict['client'].groupby(by=['county_map', 'is_business']).eic_count.sum().unstack().rename(columns={0:'not_business', 1:'business'})

plt.figure(figsize=(10, 6))
plt.bar(df.index, df.not_business, width=0.4, align='center', label='Not Business')
plt.bar(df.index, df.business, width=0.4, align='edge', label='Business')

plt.xlabel('Counties')
plt.ylabel('Total Consumption Points')
plt.title('Total Consumption Points Segmented by Business / Not Business')
plt.legend()
plt.xticks(rotation=90)
plt.grid(alpha=.25)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 8))
g = sns.catplot(
    data=enefit_dict['client'],
    kind="bar",
    orient='h',
    y='county_map',
    x='eic_count',
    hue='is_business',
    col='product_type_map',
    col_wrap=2,
    height=4,
    estimator=sum
)

g.set_axis_labels('County', 'Consumption Points')
g.set_titles('Product Type: {col_name}')

new_labels = ['Non-business', 'Business']
for ax in g.axes.flat:
    handles, _ = ax.get_legend_handles_labels()
    ax.legend(handles, new_labels, loc='center left', bbox_to_anchor=(1, 0.5))

# Remove the previous legend
g._legend.remove()

plt.subplots_adjust(top=0.9)
g.fig.suptitle('Total Consumption Points by Product Type & Business Segments')

plt.show()


In [ ]:
df = enefit_dict['client'].query(
    'product_type_map=="General service"'
).groupby(by=['county_map', 'is_business']).eic_count.sum().unstack().rename(columns={0:'not_business', 1:'business'})

plt.figure(figsize=(10, 6))
plt.bar(df.index, df.not_business, width=0.4, align='center', label='Not Business')
plt.bar(df.index, df.business, width=0.4, align='edge', label='Business')

plt.xlabel('Counties')
plt.ylabel('Total Consumption Points')
plt.title('Total Consumption Points (General service) Segmented by Business / Not Business')
plt.legend()
plt.xticks(rotation=45)
plt.grid(alpha=.25)
plt.show()


In [ ]:
df = enefit_dict['client'].query(
    'product_type_map=="Combined"'
).groupby(by=['county_map', 'is_business']).eic_count.sum().unstack().rename(columns={0:'not_business', 1:'business'})
df['not_business'] = 0

plt.figure(figsize=(10, 6))
plt.bar(df.index, df.not_business, width=0.4, align='center', label='Not Business')
plt.bar(df.index, df.business, width=0.4, align='edge', label='Business')

plt.xlabel('Counties')
plt.ylabel('Total Consumption Points')
plt.title('Total Consumption Points (Combined) Segmented by Business / Not Business')
plt.legend()
plt.xticks(rotation=45)
plt.grid(alpha=.25)
plt.show()


In [ ]:
monthly_data = enefit_dict['client'].set_index('date').eic_count.resample('M').last()

# Calculate the absolute increase
monthly_data.name = monthly_data.name.strip()
month_to_month_change = monthly_data.diff()

# Calculate the average increase
avg_month_to_month_change = month_to_month_change.mean()
print(f"Average month-on-month increase in Consumption Points: {avg_month_to_month_change:.2f}")

In [ ]:
df = enefit_dict['client'].set_index('date').eic_count.resample('D').sum()
fig, ax = plt.subplots()

ax.plot(df)
ax.set_xlabel('Date')
ax.set_ylabel('Consumption Points')
ax.set_title('Total Daily Consumption Points Over Time')

fig.autofmt_xdate()
plt.tight_layout()
plt.show()

In [ ]:
sns.lineplot(
    x='date', y='eic_count', data=enefit_dict['client']
)
plt.xlabel('Date')
plt.ylabel('Consumption Points')
plt.title('Time Series Plot of Consumption Points')
plt.xticks(rotation=45)
plt.grid(alpha=.3)
plt.show()

Let's consider if eic count can be modeled using time series forecasting:

The time series for eic count has a clear trend. However, seasonality is not clear.

In [ ]:
decomposition = STL(enefit_dict['client'].set_index('date').eic_count, period=365).fit()

fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, ncols=1, sharex=True, figsize=(10,8))

ax1.plot(decomposition.observed, color=deep_colors[0])
ax1.set_ylabel('Observed')

ax2.plot(decomposition.trend, color=deep_colors[1])
ax2.set_ylabel('Trend')

ax3.plot(decomposition.seasonal, color=deep_colors[2])
ax3.set_ylabel('Seasonal')

ax4.plot(decomposition.resid, color=deep_colors[3])
ax4.set_ylabel('Residuals')

fig.autofmt_xdate()
plt.tight_layout()

plt.show()

Total EIC Count:

First order differencing to make the time series stationary:

In [ ]:
df = enefit_dict['client'].set_index('date').eic_count.resample('D').sum()
ad_fuller_result = adfuller(df)

print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

In [ ]:
df_diff = np.diff(df, n=1)
ad_fuller_result = adfuller(df_diff)

print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

EIC Count is also a random walk.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
plot_acf(df_diff, lags=10, ax=ax1)
plot_pacf(df_diff, lags=10, ax=ax2)
plt.tight_layout()
plt.show()

Average EIC Count:

In [ ]:
df = enefit_dict['client'].set_index('date').eic_count.resample('D').mean()
ad_fuller_result = adfuller(df)

print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

In [ ]:
df_diff = np.diff(df, n=1)
ad_fuller_result = adfuller(df_diff)

print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
plot_acf(df_diff, lags=10, ax=ax1)
plot_pacf(df_diff, lags=10, ax=ax2)
plt.tight_layout()
plt.show()

**Electricity Prices**

The features in this dataset are:
* origin_date: The date when the day-ahead prices became available.
* forecast_date: The date when the forecast prices should be relevant.
* euros_per_mwh: The price of electricity on the day ahead markets in euros per megawatt hour.
* data_block_id: All rows sharing the same `data_block_id` will be available at the same forecast time. This is a function of what information is available when forecasts are actually made, at 11 AM each morning. For example, if the forecast weather `data_block_id` for predictions made on October 31st is 100 then the historic weather data_block_id for October 31st will be 101 as the historic weather data is only actually available the next day.

In [ ]:
enefit_dict['electricity_prices'].head()

In [ ]:
enefit_dict['electricity_prices'].describe()

In [ ]:
plt.figure(figsize=(12, 8))
plt.subplot(121)
enefit_dict['electricity_prices'].euros_per_mwh.plot.hist(
    bins=30, 
    title='Distribution of Electricity Prices',
)
plt.grid(alpha=.5)

plt.subplot(122)
enefit_dict['electricity_prices'].euros_per_mwh.plot.box(title='Box Plot of Electricity Prices')
plt.grid(alpha=.5)

plt.tight_layout()
plt.show()

25% of the dataset is greater than the 75th percentile:

In [ ]:
enefit_dict['electricity_prices'].query('euros_per_mwh > 200').shape[0] / enefit_dict['electricity_prices'].shape[0]

There's a single large outlier record. It is for `data_block_id` 351:

In [ ]:
enefit_dict['electricity_prices'].query('euros_per_mwh > 2000')

In [ ]:
avg_cost_full = enefit_dict['electricity_prices'].query('data_block_id==351').euros_per_mwh.mean()
avg_cost_filtered = enefit_dict['electricity_prices'].query('data_block_id==351 & euros_per_mwh < 2000').euros_per_mwh.mean()
euro_symbol = '\u20AC'

print(f'The average electricity cost with outlier is {euro_symbol}{avg_cost_full:,.2f}, while the average cost without outlier is {euro_symbol}{avg_cost_filtered:,.2f}.')

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(211)
plt.plot(
    enefit_dict['electricity_prices'].forecast_date,
    enefit_dict['electricity_prices'].euros_per_mwh, 
    linestyle='-', linewidth=1
)
plt.xlabel('Date')
plt.ylabel('Electricity Price')
plt.title('Electricity Prices Over Time')

plt.subplot(212)
plt.plot(
    enefit_dict['electricity_prices'].query('euros_per_mwh < 2000').forecast_date,
    enefit_dict['electricity_prices'].query('euros_per_mwh < 2000').euros_per_mwh,
    linestyle='-', linewidth=1
)
plt.xlabel('Date')
plt.ylabel('Electricity Price')
plt.title('Electricity Prices Over Time (No Outliers)')

plt.tight_layout()
plt.show()

In [ ]:
monthly_mean = enefit_dict['electricity_prices'].query('euros_per_mwh < 2000').set_index('forecast_date').euros_per_mwh.resample('M').mean().interpolate()

plt.figure(figsize=(12, 8))
plt.subplot(211)
plt.plot(
    monthly_mean.index, monthly_mean, marker='o', linestyle='-', linewidth=1.5
)
plt.xticks(rotation=45)
plt.ylabel('Mean Electricity Price')
plt.title('Average Monthly Cost of Electricity Over Time (No Outliers)')
plt.grid(alpha=.5)

plt.subplot(212)
monthly_mean = enefit_dict['electricity_prices'].query('euros_per_mwh < 2000').euros_per_mwh.groupby(enefit_dict['electricity_prices'].forecast_date.dt.month).mean()

month_mapping = {
    1: 'Jan', 2: 'Feb', 3: 'March', 4: 'April',
    5: 'May', 6: 'June', 7: 'July', 8: 'Aug',
    9: 'Sept', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}
monthly_mean.index = monthly_mean.index.map(month_mapping)
plt.plot(
    monthly_mean.index, monthly_mean, marker='o', linestyle='-', linewidth=1.5
)
plt.xlabel('Months')
plt.ylabel('Mean Electricity Price')
plt.title('Average Electricity Price Grouped by Month (No Outliers)')
plt.grid(alpha=.5)

plt.tight_layout()
plt.show()

print('Electricity Costs are on average highest in the summer, with the peak occurring in the summer of 2022')

In [ ]:
enefit_dict['electricity_prices']['forecast_month'] = enefit_dict['electricity_prices'].forecast_date.dt.month

plt.figure(figsize=(10, 6))
sns.boxplot(
    x='forecast_month', y='euros_per_mwh', data=enefit_dict['electricity_prices']
)
# plt.grid(alpha=.5)
plt.title('Energy Prices by Month (€)')
plt.show()

In [ ]:
monthly_data = enefit_dict['electricity_prices'].set_index('forecast_date').euros_per_mwh.resample('M').last()  
monthly_data.name = monthly_data.name.strip()
month_to_month_change = monthly_data.diff()
avg_month_to_month_change = month_to_month_change.mean()
euro_symbol = '\u20AC'

print(f"Average month-on-month change in Electricity Cost: {euro_symbol}{avg_month_to_month_change:.2f}")

The cost of electricity is generally in the same range except for a significant spike in August 2022.

In [ ]:
plt.figure(figsize=(12, 8))
sns.lineplot(
    x='forecast_date', y='euros_per_mwh', 
    data=enefit_dict['electricity_prices'],
    estimator='sum', color=deep_colors[0], 
    linewidth=1.2, alpha=.8, label='Total Price', 
)
sns.lineplot(
    x='forecast_date', y='euros_per_mwh', 
    data=enefit_dict['electricity_prices'],
    estimator='mean', color=deep_colors[1], 
    linewidth=1.2, alpha=.4, label='Average Price'
)
plt.xlabel('Date')
plt.ylabel('Electricity Cost')
plt.title('Time Series Plot of Electricity Cost')
plt.xticks(rotation=45)
plt.grid(alpha=.3)
# plt.legend().remove()

plt.show()

Time Series Plot without Outlier record:

In [ ]:
plt.figure(figsize=(12, 8))
sns.lineplot(
    x='forecast_date', y='euros_per_mwh', 
    data=enefit_dict['electricity_prices'].query('euros_per_mwh < 2000'),
    estimator='sum', linewidth=1.2, color=deep_colors[0], 
    alpha=.8, label='Total Price'
)
sns.lineplot(
    x='forecast_date', y='euros_per_mwh', 
    data=enefit_dict['electricity_prices'].query('euros_per_mwh < 2000'),
    estimator='mean', linewidth=1.2, color=deep_colors[1], 
    alpha=.4, label='Average Price'
)
plt.xlabel('Date')
plt.ylabel('Electricity Cost')
plt.title('Time Series Plot of Electricity Cost (Without Outliers)')
plt.xticks(rotation=45)
plt.grid(alpha=.3)
# plt.legend().remove()

plt.show()

Time Series Decomposition:

Decompose the time series into its trend, seasonality, and residual components to better understand underlying patterns.

In [ ]:
# 24H Period
decomposition = STL(enefit_dict['electricity_prices'].set_index('forecast_date').euros_per_mwh, period=24).fit()
observed = decomposition.observed
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Plot the original time series
plt.figure(figsize=(15, 15))
plt.subplot(411)
plt.plot(observed, label='Observed', color=deep_colors[0])
plt.legend(loc='upper left')
plt.title('Original Time Series')

# Plot the trend component
plt.subplot(412)
plt.plot(trend, label='Trend', color=deep_colors[1])
plt.legend(loc='upper left')
plt.title('Trend Component')

# Plot the seasonal component
plt.subplot(413)
plt.plot(seasonal, label='Seasonal', color=deep_colors[2])
plt.legend(loc='upper left')
plt.title('Seasonal Component')

# Plot the residual component
plt.subplot(414)
plt.plot(residual, label='Residual', color=deep_colors[3])
plt.legend(loc='upper left')
plt.title('Residual Component')

plt.tight_layout()
plt.show()

In [ ]:
# 365D Period
decomposition = STL(enefit_dict['electricity_prices'].set_index('forecast_date').euros_per_mwh, period=365*24).fit()
observed2 = decomposition.observed
trend2 = decomposition.trend
seasonal2 = decomposition.seasonal
residual2 = decomposition.resid

# Plot the original time series
plt.figure(figsize=(15, 15))
plt.subplot(411)
plt.plot(observed2, label='Observed', color=deep_colors[0])
plt.legend(loc='upper left')
plt.title('Original Time Series')

# Plot the trend component
plt.subplot(412)
plt.plot(trend2, label='Trend', color=deep_colors[1])
plt.legend(loc='upper left')
plt.title('Trend Component')

# Plot the seasonal component
plt.subplot(413)
plt.plot(seasonal2, label='Seasonal', color=deep_colors[2])
plt.legend(loc='upper left')
plt.title('Seasonal Component')

# Plot the residual component
plt.subplot(414)
plt.plot(residual2, label='Residual', color=deep_colors[3])
plt.legend(loc='upper left')
plt.title('Residual Component')

plt.tight_layout()
plt.show()

In [ ]:
# 30D Period
decomposition = STL(enefit_dict['electricity_prices'].set_index('forecast_date').euros_per_mwh, period=30*24).fit()
observed3 = decomposition.observed
trend3 = decomposition.trend
seasonal3 = decomposition.seasonal
residual3 = decomposition.resid

# Plot the original time series
plt.figure(figsize=(15, 15))
plt.subplot(411)
plt.plot(observed3, label='Observed', color=deep_colors[0])
plt.legend(loc='upper left')
plt.title('Original Time Series')

# Plot the trend component
plt.subplot(412)
plt.plot(trend3, label='Trend', color=deep_colors[1])
plt.legend(loc='upper left')
plt.title('Trend Component')

# Plot the seasonal component
plt.subplot(413)
plt.plot(seasonal3, label='Seasonal', color=deep_colors[2])
plt.legend(loc='upper left')
plt.title('Seasonal Component')

# Plot the residual component
plt.subplot(414)
plt.plot(residual3, label='Residual', color=deep_colors[3])
plt.legend(loc='upper left')
plt.title('Residual Component')

plt.tight_layout()
plt.show()

Analyzing Trend Component:

There are erratic movements in the trend which might suggest volatility, however, the trendline is flat, indicating that there is no overall trend or movement in the price of electricity. Prices remain constant with no discernible change or variation over the observed time period:

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12,8), sharex=True)

ax1.plot(trend, label='Trend', color=deep_colors[1])
# Fit a linear regression trend-line
x_values = pd.to_numeric(trend.index) / 10**9
coefficients = np.polyfit(x_values, trend.fillna(0), 1)
slope, intercept = coefficients
trendline_values = slope * x_values + intercept
trendline_dates = pd.to_datetime(x_values.astype(int) * 10**9)

ax1.plot(trendline_dates, trendline_values, label='Trendline', linestyle='--', color='red')
ax1.legend(loc='upper left')
ax1.set_title('Trend Component (24H)')


ax2.plot(trend2, label='Trend', color=deep_colors[1])
# Fit a linear regression trend-line
x_values = pd.to_numeric(trend2.index) / 10**9
coefficients = np.polyfit(x_values, trend2.fillna(0), 1)
slope, intercept = coefficients
trendline_values = slope * x_values + intercept
trendline_dates = pd.to_datetime(x_values.astype(int) * 10**9)

ax2.plot(trendline_dates, trendline_values, label='Trendline', linestyle='--', color='red')
# ax2.legend(loc='upper left')
ax2.set_title('Trend Component (1Y)')


ax3.plot(trend3, label='Trend', color=deep_colors[1])
# Fit a linear regression trend-line
x_values = pd.to_numeric(trend3.index) / 10**9
coefficients = np.polyfit(x_values, trend3.fillna(0), 1)
slope, intercept = coefficients
trendline_values = slope * x_values + intercept
trendline_dates = pd.to_datetime(x_values.astype(int) * 10**9)

ax3.plot(trendline_dates, trendline_values, label='Trendline', linestyle='--', color='red')
# ax3.legend(loc='upper left')
ax3.set_title('Trend Component (30D)')

plt.tight_layout()
plt.show()

Analyzing Seasonal Component:

When we drill down into the seasonal component, we find repetitive and predictitve patterns occurring at fixed intervals within the data. Seasonality represents the systematic, periodic fluctuations that repeat over specific time periods (e.g., daily, weekly, monthly, yearly).

For electricity prices, we have repetitive weekly fluctuations throughout the data:

In [ ]:
year = 2022
months_to_select = [1]  

selected_data = seasonal[(seasonal.index.year == year) & (seasonal.index.month.isin(months_to_select))]
selected_data2 = seasonal2[(seasonal2.index.year == year) & (seasonal2.index.month.isin(months_to_select))]
selected_data3 = seasonal3[(seasonal3.index.year == year) & (seasonal3.index.month.isin(months_to_select))]

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 12), sharex=True)

ax1.plot(selected_data, label='Seasonal', color=deep_colors[2])
ax1.set_title(f'24H Seasonal Trends for {year} - Jan')
ax1.legend(loc='upper left')

ax2.plot(selected_data2, label='Seasonal', color=deep_colors[2])
ax2.set_title(f'1Y Seasonal Trends for {year} - Jan')
ax2.set_xlabel('Date')

ax3.plot(selected_data2, label='Seasonal', color=deep_colors[2])
ax3.set_title(f'30D Seasonal Trends for {year} - Jan')
ax3.set_xlabel('Date')

fig.autofmt_xdate()
plt.tight_layout()
plt.show()

Assessing the amplitude or magnitude of the seasonal fluctuations:

The range and standard deviation of the seasonal component are not as high, indicating that the amplitude are not as high and there is not significant variability fo data points from the mean.

However, the Coefficient of variation is significantly large indicating that the mean of seasonality is small relative to the variability or dispersion of the data points (or the spread or variability among the data points is large relative to that mean).

In [ ]:
# Assessing amplitude using statistical measures
seasonal_amplitude = seasonal.max() - seasonal.min()
std_dev = seasonal.std()
coefficient_of_variation = std_dev / seasonal.mean()
amplitude_mean = seasonal.mean()
amplitude_median = seasonal.median()
amplitude_variance = seasonal.var()
amplitude_percentile_95 = np.percentile(seasonal, 95)
amplitude_percentile_5 = np.percentile(seasonal, 5)

print("24H Seasonal Component")
print(f"Amplitude (Range) of Seasonal Component: {seasonal_amplitude:.2f}")
print(f"Mean of Seasonal Component: {amplitude_mean}")
print(f"Median of Seasonal Component: {amplitude_median:.2f}")
print(f"Variance of Seasonal Component: {amplitude_variance:.2f}")
print(f"Standard Deviation of Seasonal Component: {std_dev:.2f}")
print(f"Coefficient of Variation of Seasonal Component: {coefficient_of_variation:.4f}")
print(f"95th Percentile of Seasonal Component: {amplitude_percentile_95:.2f}")
print(f"5th Percentile of Seasonal Component: {amplitude_percentile_5:.2f}")

In [ ]:
# Assessing amplitude using statistical measures
seasonal_amplitude = seasonal2.max() - seasonal2.min()
std_dev = seasonal2.std()
coefficient_of_variation = std_dev / seasonal2.mean()
amplitude_mean = seasonal2.mean()
amplitude_median = seasonal2.median()
amplitude_variance = seasonal2.var()
amplitude_percentile_95 = np.percentile(seasonal2, 95)
amplitude_percentile_5 = np.percentile(seasonal2, 5)

print("1Y Seasonal Component")
print(f"Amplitude (Range) of Seasonal Component: {seasonal_amplitude:.2f}")
print(f"Mean of Seasonal Component: {amplitude_mean}")
print(f"Median of Seasonal Component: {amplitude_median:.2f}")
print(f"Variance of Seasonal Component: {amplitude_variance:.2f}")
print(f"Standard Deviation of Seasonal Component: {std_dev:.2f}")
print(f"Coefficient of Variation of Seasonal Component: {coefficient_of_variation:.4f}")
print(f"95th Percentile of Seasonal Component: {amplitude_percentile_95:.2f}")
print(f"5th Percentile of Seasonal Component: {amplitude_percentile_5:.2f}")

In [ ]:
# Assessing amplitude using statistical measures
seasonal_amplitude = seasonal3.max() - seasonal3.min()
std_dev = seasonal3.std()
coefficient_of_variation = std_dev / seasonal3.mean()
amplitude_mean = seasonal3.mean()
amplitude_median = seasonal3.median()
amplitude_variance = seasonal3.var()
amplitude_percentile_95 = np.percentile(seasonal3, 95)
amplitude_percentile_5 = np.percentile(seasonal3, 5)

print("30D Seasonal Component")
print(f"Amplitude (Range) of Seasonal Component: {seasonal_amplitude:.2f}")
print(f"Mean of Seasonal Component: {amplitude_mean}")
print(f"Median of Seasonal Component: {amplitude_median:.2f}")
print(f"Variance of Seasonal Component: {amplitude_variance:.2f}")
print(f"Standard Deviation of Seasonal Component: {std_dev:.2f}")
print(f"Coefficient of Variation of Seasonal Component: {coefficient_of_variation:.4f}")
print(f"95th Percentile of Seasonal Component: {amplitude_percentile_95:.2f}")
print(f"5th Percentile of Seasonal Component: {amplitude_percentile_5:.2f}")

Compare the amplitude of seasonal fluctuations across different time periods or seasonal cycles:

In [ ]:
data = enefit_dict['electricity_prices'].set_index('forecast_date').euros_per_mwh.asfreq('H').bfill()

result = seasonal_decompose(data, model='additive') 

period = 24
num_cycles = len(data) // period
seasonal_amplitudes = []
for i in range(num_cycles):
    start_idx = i * period
    end_idx = min((i + 1) * period, len(data))
    seasonal_component = result.seasonal[start_idx:end_idx]
    amplitude = seasonal_component.max() - seasonal_component.min()
    cv = (seasonal_component.std() / seasonal_component.mean()) * 100
    seasonal_amplitudes.append({'Period': i + 1, 'Amplitude': amplitude, 'CV_24H': cv})
amplitudes_df_24 = pd.DataFrame(seasonal_amplitudes)

period = 365 * 24
num_cycles = len(data) // period
seasonal_amplitudes = []
for i in range(num_cycles):
    start_idx = i * period
    end_idx = min((i + 1) * period, len(data))
    seasonal_component = result.seasonal[start_idx:end_idx]
    amplitude = seasonal_component.max() - seasonal_component.min()
    cv = (seasonal_component.std() / seasonal_component.mean()) * 100
    seasonal_amplitudes.append({'Period': i + 1, 'Amplitude': amplitude, 'CV_365D': cv})
amplitudes_df_365 = pd.DataFrame(seasonal_amplitudes)

period = 30 * 24
num_cycles = len(data) // period
seasonal_amplitudes = []
for i in range(num_cycles):
    start_idx = i * period
    end_idx = min((i + 1) * period, len(data))
    seasonal_component = result.seasonal[start_idx:end_idx]
    amplitude = seasonal_component.max() - seasonal_component.min()
    cv = (seasonal_component.std() / seasonal_component.mean()) * 100
    seasonal_amplitudes.append({'Period': i + 1, 'Amplitude': amplitude, 'CV_30D': cv})
amplitudes_df_30 = pd.DataFrame(seasonal_amplitudes)

amplitudes_df = amplitudes_df_24.merge(amplitudes_df_365, how='left', on=['Period', 'Amplitude'])
amplitudes_df = amplitudes_df.merge(amplitudes_df_30, how='left', on=['Period', 'Amplitude'])
amplitudes_df.head()

Analyzing Residual Component:

Analyzing the residual component resulting from a seasonal decomposition helps to understand the unexplained variation in the time series after accounting for the trend, seasonality, and other identified components. It is crucial to assess the goodness-of-fit of the decomposition model and to detect any remaining patterns or information that the model didn't capture.

The distribution of the residual plot appears to be approximately normal and the autocorrelation plot shows a flat line with no significant spikes. This indicates that residuals are centered and have consistent variability. 

The flat autocorrelation also indicates the absence of significant autocorrelation at different lags, which suggests that the residuals are not correlated at different time lags, indicating randomness or lack of systematic patterns.

The normal distribution and absence of significant autocorrelation in residuals suggest that the seasonal decomposition model adequately captures the underlying trend, seasonality, and other components present in the time series data for electricty prices. This implies goodness of fit and model adequacy.

The lack of autocorrelation further implies that any remaining variation in the data captured by the residuals is random noise or unexplained variability, rather than systematic patterns or trends.

We can therefore use the decomposition model to forecast future observation as the model adequately captures the data's variability.

The best model is the 365D model.

In [ ]:
# Visualize residuals - 24H
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
residual.plot(title='Residuals Time Plot')

plt.subplot(2, 2, 2)
residual.plot(kind='hist', bins=20, title='Residuals Distribution')

plt.subplot(2, 2, 3)
residual.plot(kind='kde', title='Residuals Density Plot')

plt.subplot(2, 2, 4)
pd.plotting.autocorrelation_plot(residual)
plt.title('Autocorrelation Plot of Residuals')

plt.tight_layout()
plt.show()

In [ ]:
# Visualize residuals - 365D
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
residual2.plot(title='Residuals Time Plot')

plt.subplot(2, 2, 2)
residual2.plot(kind='hist', bins=20, title='Residuals Distribution')

plt.subplot(2, 2, 3)
residual2.plot(kind='kde', title='Residuals Density Plot')

plt.subplot(2, 2, 4)
pd.plotting.autocorrelation_plot(residual2)
plt.title('Autocorrelation Plot of Residuals')

plt.tight_layout()
plt.show()

In [ ]:
# Visualize residuals - 30D
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
residual3.plot(title='Residuals Time Plot')

plt.subplot(2, 2, 2)
residual3.plot(kind='hist', bins=20, title='Residuals Distribution')

plt.subplot(2, 2, 3)
residual3.plot(kind='kde', title='Residuals Density Plot')

plt.subplot(2, 2, 4)
pd.plotting.autocorrelation_plot(residual3)
plt.title('Autocorrelation Plot of Residuals')

plt.tight_layout()
plt.show()

In [ ]:
# Statistical Analysis
print("Summary Statistics of Residuals - 24H:")
print(residual.describe())

In [ ]:
print("Summary Statistics of Residuals - 365D:")
print(residual2.describe())

In [ ]:
print("Summary Statistics of Residuals - 30D:")
print(residual3.describe())

When we use linear regression to moddel electrictiy prices, we find that the time index is a poor predictor of price. However, 1H and 24H lags as well as 24H MA and 30D MA are better predictors of electricity price using a linear regression model.

In [ ]:
data = enefit_dict['electricity_prices'].set_index('forecast_date').euros_per_mwh.asfreq('H').bfill()
data = data.to_frame()
data['time'] = np.arange(len(data.index))
data['lag_1'] = data.euros_per_mwh.shift(1)
data['lag_12'] = data.euros_per_mwh.shift(12)
data['lag_24'] = data.euros_per_mwh.shift(24)
data['lag_30d'] = data.euros_per_mwh.shift(30*24)
data['lag_365d'] = data.euros_per_mwh.shift(365*24)
data['MA_24'] = data.euros_per_mwh.rolling(window=24, center=True, min_periods=12).mean()
data['MA_30'] = data.euros_per_mwh.rolling(window='30D', center=True, min_periods=24*15).mean()
data['MA_365'] = data.euros_per_mwh.rolling(window='365D', center=True, min_periods=24*183).mean()

data.head()

In [ ]:
formula = 'euros_per_mwh ~ time'
model = smf.ols(formula, data=data)
results = model.fit()
print(results.summary())

In [ ]:
formula = 'euros_per_mwh ~ lag_1'
model = smf.ols(formula, data=data)
results = model.fit()
print(results.summary())

In [ ]:
formula = 'euros_per_mwh ~ lag_12'
model = smf.ols(formula, data=data)
results = model.fit()
print(results.summary())

In [ ]:
formula = 'euros_per_mwh ~ lag_24'
model = smf.ols(formula, data=data)
results = model.fit()
print(results.summary())

In [ ]:
formula = 'euros_per_mwh ~ lag_30d'
model = smf.ols(formula, data=data)
results = model.fit()
print(results.summary())

In [ ]:
formula = 'euros_per_mwh ~ lag_365d'
model = smf.ols(formula, data=data)
results = model.fit()
print(results.summary())

In [ ]:
formula = 'euros_per_mwh ~ MA_24'
model = smf.ols(formula, data=data)
results = model.fit()
print(results.summary())

In [ ]:
formula = 'euros_per_mwh ~ MA_30'
model = smf.ols(formula, data=data)
results = model.fit()
print(results.summary())

In [ ]:
formula = 'euros_per_mwh ~ MA_365'
model = smf.ols(formula, data=data)
results = model.fit()
print(results.summary())

Time Series Modeling:

The time series is stationary.

In [ ]:
df = enefit_dict['electricity_prices'].set_index('forecast_date').euros_per_mwh.asfreq('H').bfill()
ad_fuller_result = adfuller(df)

print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

In testing for autocorrelation and partial autocorrelation, we find that significant coefficients exist beyond the first lag for both plots. This means we have a time series that can be modeled usign 
ARIMA.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
plot_acf(df, lags=10, ax=ax1)
plot_pacf(df, lags=10, ax=ax2)
plt.show()

With the stationary time series, we'll use an ARIMA model to determine the optimum AR(_p_) and MA(_q_) values for the dataset. 

In [ ]:
def optimize_ARIMA(
        # The order_list parameter now includes p, q orders. 
        endog: Union[pd.Series, list], 
        # exog: Union[pd.Series, list], # useful for a SARIMAX model
        order_list: list, 
        d: int, 
        # D: int, s: int # useful for a SARIMAX model
        ) -> pd.DataFrame:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        results = []
        
        # Loop over all unique ARIMA(p,d,q) models, 
        # fit them, and store the AICs.
        for order in notebook.tqdm(order_list):
            try: 
                model = SARIMAX(
                    endog, 
                    # exog,
                    order=(order[0], d, order[1]),
                    # seasonal_order=(order[2], D, order[3], s),
                    simple_differencing=False).fit(disp=False)
            except:
                continue
                
            aic = model.aic
            results.append([order, aic])
            
        result_df = pd.DataFrame(results)
        result_df.columns = ['(p,q)', 'AIC']
        
        #Sort in ascending order, lower AIC is better
        result_df = result_df.sort_values(by='AIC', ascending=True).reset_index(drop=True)
        
        return result_df

In [ ]:
ps = range(0, 5, 1)
qs = range(0, 5, 1)
d = 0

ARIMA_order_list = list(product(ps, qs))
ARIMA_result_df = optimize_ARIMA(df, ARIMA_order_list, d)
ARIMA_result_df.head()

In [ ]:
ps = range(0, 5, 1)
qs = range(0, 5, 1)
d = 1

ARIMA_order_list = list(product(ps, qs))
ARIMA_result_df = optimize_ARIMA(df, ARIMA_order_list, d)
ARIMA_result_df.head()

This model is based on data that is recorded hourly. We'll resample to daily records and check the AICs for the model:

Total Daily Price:

Test for stationarity:

In [ ]:
df = enefit_dict['electricity_prices'].set_index('forecast_date').euros_per_mwh.resample('D').sum()

ad_fuller_result = adfuller(df)
print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

In [ ]:
df_diff = np.diff(df, n=1)
ad_fuller_result = adfuller(df_diff)
print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

Test for autocorrelation and partial autocorrelation:

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
plot_acf(df, lags=10, ax=ax1)
plot_pacf(df, lags=10, ax=ax2)
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
plot_acf(df_diff, lags=10, ax=ax1)
plot_pacf(df_diff, lags=10, ax=ax2)
plt.show()

Model data and select best AIC:

In [ ]:
ps = range(0, 10, 1)
qs = range(0, 10, 1)
d = 1

ARIMA_order_list = list(product(ps, qs))
ARIMA_result_df = optimize_ARIMA(df, ARIMA_order_list, d)
ARIMA_result_df.head()

Average Daily Price:

In [ ]:
df = enefit_dict['electricity_prices'].set_index('forecast_date').euros_per_mwh.resample('D').mean()

ad_fuller_result = adfuller(df)
print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

In [ ]:
df_diff = np.diff(df, n=1)
ad_fuller_result = adfuller(df_diff)
print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
plot_acf(df, lags=10, ax=ax1)
plot_pacf(df, lags=10, ax=ax2)
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
plot_acf(df_diff, lags=10, ax=ax1)
plot_pacf(df_diff, lags=10, ax=ax2)
plt.show()

In [ ]:
ps = range(0, 10, 1)
qs = range(0, 10, 1)
d = 1

ARIMA_order_list = list(product(ps, qs))
ARIMA_result_df = optimize_ARIMA(df, ARIMA_order_list, d)
ARIMA_result_df.head()

The best model is the one on the data that has been resampled to daily records and averaged.

Fit based on best parameters:

In [ ]:
df = enefit_dict['electricity_prices'].set_index('forecast_date').euros_per_mwh.resample('D').mean()

d = 1
SARIMAX_model = SARIMAX(
    endog=df, 
    order=(ARIMA_result_df.iloc[0,0][0], d, ARIMA_result_df.iloc[0,0][1]),
    simple_differencing=False
    )
SARIMAX_model_fit = SARIMAX_model.fit(disp=False)
print(SARIMAX_model_fit.summary())

Residual Analysis:

In [ ]:
SARIMAX_model_fit.plot_diagnostics(figsize=(10,8))
plt.tight_layout()
plt.show()

In [ ]:
residuals = SARIMAX_model_fit.resid
lb_df = acorr_ljungbox(residuals, np.arange(1, 11, 1))

lb_df

Electricity prices can be modeled using ARIMA. 